In [1]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model


def Ask_nurse_name():
    num_nurses = int(input("Enter number of nurses :"))
    all_nurses = range(num_nurses)
    nurse_names = []
    for n in all_nurses:
        nurse_names.append(input("Enter name of nurse number " + str(n+1) + " :"))
    return nurse_names



def main():
    # Data.
    nurse_names = []
    nurse_names = Ask_nurse_name()
    
    num_nurses = len(nurse_names)
    num_shifts = 1
    num_days = 29
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one nurse in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == 1)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)
            
    

   
    #Shift distribution based on ranks 
    # see https://docs.google.com/spreadsheets/d/1OsIMVJO8hlFbkvxDZ9C11IDY7806Zo2YwZpz5cG3Tzw/edit#gid=0
    nurse_rank = []
    nurse_rounded_shift = []
    shift_per_nurse = []
    total_rank = 0
    total_shift_rounded = 0
    
    for n in all_nurses:
        nurse_rank.append(input("Enter rank for nurse number " + str(n) +" (" + nurse_names[n]+")"))
        total_rank = total_rank + int(nurse_rank[n])
        
    print("Total rank = " + str(total_rank))
    
    for n in all_nurses:
        nurse_rounded_shift.append(round(num_shifts*num_days/total_rank*int(nurse_rank[n])))
        total_shift_rounded = total_shift_rounded + round(num_shifts*num_days/total_rank*int(nurse_rank[n]))
    
    nurse_rounded_shift[num_nurses-1] = round(num_shifts*num_days/total_rank*int(nurse_rank[num_nurses-1]))+num_shifts*num_days-total_shift_rounded
    
    print("Total rounded shifts : " + str(total_shift_rounded))
    
    for n in all_nurses:
        print("Number of shifts for nurse number " + str(n) + " = " + str(nurse_rounded_shift[n]))
        num_shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked.append(shifts[(n, d, s)])
        model.Add(sum(num_shifts_worked) <= nurse_rounded_shift[n])

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True


    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print('Solution %i' % self._solution_count)
            for d in range(self._num_days):
                print('Day %i' % d)
                for n in range(self._num_nurses):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.Value(self._shifts[(n, d, s)]):
                            is_working = True
                            print('  Nurse %i works shift %i' % (n, s))
                    if not is_working:
                        print('  Nurse {} does not work'.format(n))
            if self._solution_count >= self._solution_limit:
                print('Stop search after %i solutions' % self._solution_limit)
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(shifts, num_nurses,
                                                    num_days, num_shifts,
                                                    solution_limit)

    solver.Solve(model, solution_printer)

    # Statistics.
    print('\nStatistics')
    print('  - conflicts      : %i' % solver.NumConflicts())
    print('  - branches       : %i' % solver.NumBranches())
    print('  - wall time      : %f s' % solver.WallTime())
    print('  - solutions found: %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Enter number of nurses : 2
Enter name of nurse number 1 : Serra
Enter name of nurse number 2 : Kent
Enter rank for nurse number 0 (Serra) 1
Enter rank for nurse number 1 (Kent) 2


Total rank = 3
Total rounded shifts : 29
Number of shifts for nurse number 0 = 10
Number of shifts for nurse number 1 = 19
Solution 1
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
Day 1
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 2
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 3
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 4
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 5
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 6
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 7
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 8
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 9
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 10
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 11
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 12
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 13
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 14
  Nurse 0 does not work
  Nurse 1 works shift 0
Day 15
  Nurse 0 does not work
  Nurse 1 works shif

In [ ]:
he
